In [3]:
# =========================
# AutoML for NO2 Forecasting (sklearn only)
# =========================

import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import joblib

# -------------------------
# Config (edit here)
# -------------------------
CFG = {
    "raw_csv": "data/raw/combined_iasi_no2_meteo_2020_2025_local.csv",  # <-- path to your raw combined CSV (AQ + meteo)
    "datetime_col": "datetime",
    "target_col": "value",            # target NO2
    "tz": "Europe/Bucharest",         # local timezone
    "handle_outliers": "clip",        # "clip", "midpoint", or None
    "y_min": 0,
    "y_max": 300,
    "n_splits": 3,                    # TimeSeriesSplit
    "random_search_iters": 25,
    "random_state": 42,
    "out_dir": "outputs/reports",
}

Path(CFG["out_dir"]).mkdir(parents=True, exist_ok=True)

# -------------------------
# Helpers
# -------------------------

def parse_datetime_local(df: pd.DataFrame, col: str, tz: str) -> pd.Series:
    """Parse tz-aware strings safely, normalize to target tz."""
    s = pd.to_datetime(df[col], errors="coerce", utc=True).dt.tz_convert(tz)
    return s

def make_dt_block(datetime_col="datetime", tz="Europe/Bucharest"):
    def _to_df(X):
        if isinstance(X, pd.DataFrame):
            return X
        return pd.DataFrame({datetime_col: np.asarray(X).ravel()})
    def _extract(X):
        X = _to_df(X)
        s = pd.to_datetime(X[datetime_col], errors="coerce", utc=True).dt.tz_convert(tz)
        return pd.DataFrame({
            "dt__hour": s.dt.hour,
            "dt__dow": s.dt.dayofweek,
            "dt__month": s.dt.month,
            "dt__is_weekend": (s.dt.dayofweek >= 5).astype(int),
        }, index=X.index)
    def _names(_, input_features=None):
        return np.array(["dt__hour","dt__dow","dt__month","dt__is_weekend"])
    return FunctionTransformer(_extract, feature_names_out=_names)

class SimpleTypeCoercer(BaseEstimator, TransformerMixin):
    """Coerce a subset of columns to desired dtypes (best-effort)."""
    def __init__(self, conversion_dict: dict[str, str]):
        self.conversion_dict = conversion_dict
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        Xc = X.copy()
        for col, dtype in self.conversion_dict.items():
            if col in Xc.columns:
                try: Xc[col] = Xc[col].astype(dtype, errors="ignore")
                except Exception: pass
        return Xc

def handle_y_outliers(y: pd.Series, mode: str | None, lo: float, hi: float) -> pd.Series:
    """Apply outlier handling to target."""
    if mode is None:
        return pd.to_numeric(y, errors="coerce")
    y = pd.to_numeric(y, errors="coerce")
    if mode == "clip":
        return y.clip(lower=lo, upper=hi)
    if mode == "midpoint":
        avg = (lo + hi) / 2.0
        mask = (y < lo) | (y > hi)
        y2 = y.copy()
        y2[mask] = avg
        return y2
    return y  # fallback

def build_preprocessor(datetime_col: str, tz: str):
    """Num + Cat + Datetime feature engineering"""
    # Derive selectors that exclude the raw datetime from num/cat branches
    def num_sel(X):
        cols = X.select_dtypes(include=np.number).columns.tolist()
        return [c for c in cols if c != datetime_col]
    def cat_sel(X):
        cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
        return [c for c in cols if c != datetime_col]

    num_pipe = Pipeline([
        ("imp", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    cat_pipe = Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ])
    dt_pipe = Pipeline([
        ("dt", make_dt_block(datetime_col, tz)),
        ("scaler", StandardScaler())
    ])
    pre = ColumnTransformer([
        ("num", num_pipe, num_sel),
        ("cat", cat_pipe, cat_sel),
        ("dt",  dt_pipe, [datetime_col]),
    ])
    return pre

def get_model_space(random_state: int):
    """Define candidate models & hyperparam dists for RandomizedSearchCV."""
    space = []

    # Linear baselines
    space.append((
        "LinearRegression", LinearRegression(), 
        {"fit_intercept": [True, False]}
    ))

    space.append((
        "Ridge", Ridge(random_state=random_state),
        {"alpha": np.logspace(-3, 3, 30)}
    ))

    space.append((
        "Lasso", Lasso(random_state=random_state, max_iter=5000),
        {"alpha": np.logspace(-3, 1, 20)}
    ))

    # Trees / boosting
    space.append((
        "RandomForest", RandomForestRegressor(random_state=random_state, n_jobs=-1),
        {
            "n_estimators": [150, 300, 500],
            "max_depth": [None, 6, 10, 14],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
            "max_features": ["sqrt", "log2", 0.5, 0.8]
        }
    ))

    space.append((
        "GradientBoosting", GradientBoostingRegressor(random_state=random_state),
        {
            "n_estimators": [150, 300, 500],
            "learning_rate": [0.03, 0.05, 0.1],
            "max_depth": [2, 3, 4],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
            "subsample": [0.7, 0.9, 1.0]
        }
    ))

    space.append((
        "HistGB", HistGradientBoostingRegressor(random_state=random_state),
        {
            "max_depth": [None, 6, 10],
            "learning_rate": [0.03, 0.05, 0.1],
            "min_samples_leaf": [10, 20, 30],
            "l2_regularization": [0.0, 0.01, 0.1]
        }
    ))

    return space

def evaluate_split(y_true, y_pred):
    mse  = mean_squared_error(y_true, y_pred)   # no `squared` kwarg
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    return rmse, mae, r2

# -------------------------
# Load + prepare raw data
# -------------------------
raw = pd.read_csv(CFG["raw_csv"])

# ensure datetime exists and target exists
assert CFG["datetime_col"] in raw.columns, f"Missing {CFG['datetime_col']}"
assert CFG["target_col"]   in raw.columns, f"Missing {CFG['target_col']}"

# normalize datetime and sort
raw[CFG["datetime_col"]] = parse_datetime_local(raw, CFG["datetime_col"], CFG["tz"])
raw = raw.sort_values(CFG["datetime_col"]).reset_index(drop=True)

# split X/y
y_raw = raw[CFG["target_col"]]
X_raw = raw.drop(columns=[CFG["target_col"]])

# target handling
y_all = handle_y_outliers(y_raw, CFG["handle_outliers"], CFG["y_min"], CFG["y_max"])

# ensure numeric and drop rows where y is NaN
y_all = pd.to_numeric(y_all, errors="coerce")
mask = y_all.notna()
n_dropped = (~mask).sum()
if n_dropped:
    print(f"[WARN] Dropping {n_dropped} rows with NaN targets before CV.")
X_raw = X_raw.loc[mask].reset_index(drop=True)
y_all = y_all.loc[mask].reset_index(drop=True)

# -------------------------
# Build preprocessing
# -------------------------
# optional: coerce common column types (edit as needed)
conversion_dict = {
    "location_id": "Int64",
    "sensors_id":  "Int64",
    "location":    "category",
    "lat":         "float64",
    "lon":         "float64",
    "parameter":   "category",
    "units":       "category",
    "temp_C":      "float64",
    "dewpoint_C":  "float64",
    "slp_hPa":     "float64",
    "wind_dir_deg":"float64",
    "wind_speed_ms":"float64",
    "sky_cover":   "category",
    "precip_mm":   "float64",
}
coercer = SimpleTypeCoercer(conversion_dict)
pre = build_preprocessor(CFG["datetime_col"], CFG["tz"])

# full preprocessing pipeline (no model yet)
preprocess = Pipeline([
    ("coerce", coercer),
    ("pre", pre),
])

# transform once to inspect feature space (fit on all is fine just for preview)
Xt_all = preprocess.fit_transform(X_raw)
feat_names = preprocess.named_steps["pre"].get_feature_names_out()
print(f"Feature matrix shape: {Xt_all.shape}")

# -------------------------
# TimeSeriesSplit & model search
# -------------------------
tscv = TimeSeriesSplit(n_splits=CFG["n_splits"])

results_rows = []
best_overall = {"score": np.inf, "name": None, "model": None}

for model_name, base_est, param_dist in get_model_space(CFG["random_state"]):
    print(f"\n=== Tuning {model_name} ===")
    # Build a full pipeline: preprocess -> estimator
    pipe = Pipeline([
        ("prep", preprocess),
        ("est", base_est)
    ])
    # Search on last split scoring = neg RMSE
    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions={f"est__{k}": v for k, v in param_dist.items()},
        n_iter=CFG["random_search_iters"],
        cv=tscv,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1,
        verbose=0,
        random_state=CFG["random_state"],
        refit=True,
    )
    search.fit(X_raw, y_all)
    best_rmse = -search.best_score_
    best_est  = search.best_estimator_

    # Evaluate per fold with the fitted best to get RMSE/MAE/R2
    fold_metrics = []
    for i, (tr, te) in enumerate(tscv.split(X_raw)):
        Xtr, Xte = X_raw.iloc[tr], X_raw.iloc[te]
        ytr, yte = y_all.iloc[tr], y_all.iloc[te]
        # fit a clone on each fold to be fair (or reuse best_est.fit if preferred)
        best_est.fit(Xtr, ytr)
        yhat = best_est.predict(Xte)
        rmse, mae, r2 = evaluate_split(yte, yhat)
        fold_metrics.append((rmse, mae, r2))

    rmse_mean = np.mean([m[0] for m in fold_metrics])
    mae_mean  = np.mean([m[1] for m in fold_metrics])
    r2_mean   = np.mean([m[2] for m in fold_metrics])

    results_rows.append({
        "Model": model_name,
        "Best_CV_RMSE": round(best_rmse, 3),
        "RMSE_mean": round(rmse_mean, 3),
        "MAE_mean": round(mae_mean, 3),
        "R2_mean": round(r2_mean, 3),
        "Best_Params": search.best_params_
    })

    # Track overall best by mean RMSE
    if rmse_mean < best_overall["score"]:
        best_overall = {"score": rmse_mean, "name": model_name, "model": search.best_estimator_}

# -------------------------
# Results & saving
# -------------------------
summary = pd.DataFrame(results_rows).sort_values("RMSE_mean")
print("\n=== AutoML Summary (lower RMSE is better) ===")
display(summary)

summary_path = Path(CFG["out_dir"]) / "automl_summary.csv"
summary.to_csv(summary_path, index=False)
print(f"Saved summary -> {summary_path}")

best_path = Path(CFG["out_dir"]) / f"best_model_{best_overall['name']}.joblib"
joblib.dump(best_overall["model"], best_path)
print(f"Saved best model -> {best_path}")

# Optional: quick baseline comparison (mean predictor)
y_mean_pred = np.full_like(y_all, fill_value=y_all.mean(), dtype=float)
rmse_baseline = mean_squared_error(y_all, y_mean_pred)**0.5
print(f"\nBaseline (mean) RMSE on full series: {rmse_baseline:.3f}")

[WARN] Dropping 27137 rows with NaN targets before CV.
Feature matrix shape: (22081, 29)

=== Tuning LinearRegression ===


/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_b


=== Tuning Ridge ===


/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_b


=== Tuning Lasso ===


...

/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['wind_speed_ms']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/nix/store/2psxf7a7lra916jd0wbafviqz53fb7kj-python3-3.13.5-env/lib/python3.13/site-packages/sklearn/impute/_b


=== AutoML Summary (lower RMSE is better) ===


,Model,Best_CV_RMSE,RMSE_mean,MAE_mean,R2_mean,Best_Params
0,LinearRegression,25.947,25.947,20.250,-1.040,{'est__fit_intercept': True}
2,Lasso,26.553,26.553,21.596,-1.047,{'est__alpha': 1.438449888287663}
1,Ridge,26.725,26.725,21.802,-1.064,{'est__alpha': 1000.0}
3,RandomForest,27.941,27.941,22.826,-1.225,"{'est__n_estimators': 300, 'est__min_samples_s..."
5,HistGB,29.075,29.075,23.639,-1.426,"{'est__min_samples_leaf': 30, 'est__max_depth'..."
4,GradientBoosting,29.234,29.234,23.077,-1.348,"{'est__subsample': 1.0, 'est__n_estimators': 1..."


Saved summary -> outputs/reports/automl_summary.csv


PicklingError: Can't pickle <function build_preprocessor.<locals>.num_sel at 0x1065004a0>: it's not found as __main__.build_preprocessor.<locals>.num_sel